In [38]:
import os 
import time
import re
import pandas as pd 
import spacy
from datetime import date

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options

nlp = spacy.load('en_core_web_sm', disable=['parser', 'lemmatizer', 'tagger'])

# if i scrape NPR The head needs to be on 

In [ ]:
#from selenium.webdriver.support.ui import Select

#def search_archive(driver):

#    select = Select(driver.find_element(By.CSS_SELECTOR, 'div.month:nth-child(2) > ul:nth-child(2)'))
    
#    select.select_by_value('1')

In [50]:
# i need a make dir function     if not os.path.exists(/home/stephbuon/projects/entascope/results/ + ts):
        #os.mkdir(export_path + ts)

def import_existing_citations(existing_set, source):
    if existing_set == True:
        scraped_pages_df = pd.read_csv('/home/stephbuon/projects/entascope/data/scraped_pages/' + ts + '/' + 'citations_' + source + '_' + ts + '.csv')
        scraped_pages_df['url'] = scraped_pages_df['url'].apply(lambda x: x.split('/')[-1])
        scraped_pages = dict(zip(scraped_pages_df['url'], scraped_pages_df['source']))
        
        return scraped_pages # do max_no elsewhere 

    else:
        return None


def start_selenium(source, url):
    options = Options()
    #options.headless = True
    driver = webdriver.Firefox(executable_path='/home/stephbuon/projects/entascope/geckodriver', options=options)
    driver.get(url)

    if source == 'FOX':
        #search_archive(driver)
        
        #WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.load-more > a:nth-child(1)'))).click() # get rid of this for media -- or maybe i Have no need at all 
        for i in range(100):
            try:
                driver.find_element(By.CSS_SELECTOR, '.pf-widget-close').click()
            except:
                pass

            try:
                driver.execute_script("arguments[0].scrollIntoView();", WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.load-more > a:nth-child(1)'))))
                WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.load-more > a:nth-child(1)'))).click()
            except Exception as e2: 
                print(e2)
                print('click number ' + str(i))
                pass

        return driver.page_source

    if source == 'NPR':
        scroll_number = 5
        for i in range(1, scroll_number):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)

            try:
                driver.find_element(By.CSS_SELECTOR, '.global-modal__dismiss').click()
            except:
                pass

        return driver.page_source

    if source == 'MSNBC':
        return driver.page_source


def validate_url(url, page_url, source):
    if source == 'FOX':
        if not 'https' in page_url:
            if '/politics/' in page_url or '/media/' in page_url or '/us/':
                page_url = 'https://www.foxnews.com' + page_url            # make sure there are no doubles 
                try:
                    return requests.get(page_url)
                except Exception as e3:
                    print(e3)
                    return None
                    
        elif 'q=coronavirus' in url:
            return requests.get(page_url)
    
    elif source == 'NPR':
        #if not 'ondemand.npr.org' and 'mp3?' in page_url:
        try:
            return requests.get(page_url)
        except Exception as e4:
            print(e4)
            return None

    else:
        try:
            return requests.get(page_url)
        except Exception as e4:
                    print(e4)
                    return None


def export_data(export_path, scraped_pages, out, source, ts):

    if not os.path.exists(export_path + ts):
        os.mkdir(export_path + ts)

    citations = pd.DataFrame(
    {'url': scraped_pages.keys(), 'source': scraped_pages.values()} )

    clean_stuff = pd.DataFrame(citations['source'].tolist(), columns=['source', 'article_id'])

    del(citations['source'])
    citations = pd.concat([citations, clean_stuff], axis=1)

    citations.to_csv(export_path + ts + '/' + 'citations_' + source + '_' + ts + '.csv', index = False)


    named_entites = pd.DataFrame(
        {'entity_label': out.keys(), 'entity_text': out.values() } )
            
    named_entites = named_entites.explode('entity_text')

    if not os.path.isfile(export_path + ts + '/' + 'named_entities_' + source + '_' + ts + '.csv'):
        named_entites.to_csv(export_path + ts + '/' + 'named_entities_' + source + '_' + ts + '.csv')
    else:
        old_named_entities = pd.read_csv(export_path + ts + '/' + 'named_entities_' + source + '_' + ts + '.csv')
        named_entites = pd.concat([named_entites, old_named_entities], ignore_index = True)

        named_entites.to_csv(export_path + ts + '/' + 'named_entities_' + source + '_' + ts + '.csv', index = False)


# NORP
#normberg law 
def show_entity_labels():
    print("""
    PERSON:        People's names
    NORP:          Nationalities or religiouse or political groups.
    FAC:           Infrastructure such as buildings, airports, highways, bridges, etc.
    ORG:           Companies, agencies, institutions, etc.
    GPE:           Countries, cities, states.
    LOC:           Non-GPE locations, mountain ranges, bodies of water. 
    PRODUCT:       Objects, vehicles, foods, etc. (Not services.)
    EVENT:         Names of events. Natural disasters, battles, sports events, etc. Coronavirus strains like "omicron" are categorized as events.
    WORK_OF_ART:   Titles of books, songs, etc.
    LAW:           Names of documents made into laws.
    LANGUAGE:      Named languages.
    DATE:          Absolute or relative dates or periods.
    TIME:          Times smaller than a day.
    PERCENT:       Percentage, including "%".
    MONEY:         Monetary values.
    QUANTITY:      Measurements, as of weight or distance.
    ORDINAL:       Terms denoting order, such as "first" or "second."
    CARDINAL:      Numerals that do not fall under another time. 
    """)




def extract_named_entities(requests_url):
    ne_dictionary = {}

    if requests_url is not None:
        try:
            page_content = BeautifulSoup(requests_url.text, 'html.parser')
            
            for paragraph in page_content.findAll('p'):
                paragraph = list(paragraph.stripped_strings)
                
                for sentence in paragraph:
                    sentence = nlp(sentence)
                    
                    for entity in sentence.ents:
                        if not entity.label_ in ne_dictionary.keys():
                            ne_dictionary[entity.label_] = [entity.text]
                        else:
                            ne_dictionary[entity.label_].append(entity.text)
        
        except Exception as e5:
            print(e5)
    
    return ne_dictionary



def time_stamp():
    today = date.today()
    d_today = today.strftime("%m-%d-%Y")

    return d_today


def scraper(url, response, source, ts, **kwargs):

    keyword = re.compile(r'covid|coronavirus|vaccine|omicron|delta|vaccinat|vaxx|pandemic|mask', re.IGNORECASE)

    start_page = BeautifulSoup(response, 'html.parser')
    
    if kwargs.get('existing_set') == True:
        scraped_pages_df = pd.read_csv('/home/stephbuon/projects/entascope/scraped_pages/' + ts + '/' + 'citations_' + source + '_' + ts + '.csv')
        scraped_pages_df['url'] = scraped_pages_df['url'].apply(lambda x: x.split('/')[-1])

        scraped_pages = dict(zip(scraped_pages_df['url'], scraped_pages_df['source']))
        
        max_no = scraped_pages_df['article_id'].max()

    else:
        scraped_pages = {}
        max_no = None

    

    #scraped_pages = import_existing_citations(source, citaitons)
    #if scraped_pages is not None:
    #    max_no = '' #get article ID max
    #else:
    #    max_no = None


    out = {}

    article = 0
    cycle = 0
    for link in start_page.findAll('a'):

        if 'href' in str(link):
            page_url = link['href']

            if keyword.search(page_url):

                if not page_url.split('/')[-1] in scraped_pages.keys():

                    ###### I can probably get rid fo recorded pages dict now that I have a method for splitting url
                    # and then I can just record the vlidated version od scraped pages  
                    cycle = cycle + 1
                    if max_no is not None:
                        if cycle == 1:
                            article = max_no + 1
                        else:
                            article = article + 1
                    else:
                        article = article + 1

                    if not 'http' in page_url and source == 'FOX':
                        scraped_pages['https://www.foxnews.com' + page_url] = [source, article] 
                    else: 
                        scraped_pages[page_url] = [source, article]

                    ### this si citations 
                  
                
                # make a distinction between citations and named entity extraction 

                    next_page = validate_url(url, page_url, source) 

                    di = extract_named_entities(next_page)
                    
                    for key in di.keys():
                        if key in di: 
                            out.setdefault(key, []).extend(di[key])                        
                            
                            
                    next_page = requests.get(page_url)
                    if next_page is not None: # should be passed requests.get(url)
                        try:
                            page_content = BeautifulSoup(next_page.text, 'html.parser')

                            for paragraph in page_content.findAll('p'):
                                paragraph = list(paragraph.stripped_strings)
                                
                                for sentence in paragraph: 
                                    try:
                                        sentence = nlp(sentence)
                                        
                                        for entity in sentence.ents:
                                            if not entity.label_ in out.keys():
                                                out[entity.label_] = [entity.text]
                                            else:
                                                out[entity.label_].append(entity.text)
                                                
                                    except Exception as e1:
                                        print(e1)
                                        
                        except Exception as e5:
                            print(e5)
                          
                    time.sleep(1)
 
   
        else:
            pass

    #start_page.quit() # will this turn off driver?
    export_data('/home/stephbuon/projects/entascope/data/scraped_pages/', scraped_pages, out, source, ts)

# class news_scraper.dynamic_load()

def scrape_pages_load_more(url, source, ts, **kwargs): # dynamic_load_scraper():

    existing_set = kwargs.get('existing_set', None)
    
    # selenium is used to activate triggers like "show more" buttons
    # the triggered website is then assigned to response (instead of using requests)
    response = start_selenium(source, url)
    #response = requests.get(url)

    scraper(url, response, source, ts, existing_set = existing_set)


def scrape_pages_multi_page(url, source, ts): # scrape_pages_multi_page():

    keyword = re.compile(r'covid|coronavirus|vaccine|omicron|delta|vaccinat|vaxx|pandemic|mask', re.IGNORECASE)
    
    options = Options()
    #options.headless = True
    driver = webdriver.Firefox(executable_path='/home/stephbuon/projects/entascope/geckodriver', options=options)

    for i in range(5): 
        if source == 'KRISTV':
            url_w_page = url + str(i)

            driver.get(url_w_page)
            
            try:
                try:
                    driver.find_element(By.CSS_SELECTOR, '.MAD_INVIEW_CLOSE').click()
                except:
                    pass

                driver.execute_script("arguments[0].scrollIntoView();", WebDriverWait(driver, .5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "a.List"))))
                WebDriverWait(driver, .5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.List"))).click()

                scraper(driver.page_source, source, ts)
                
            except Exception as e: 
                print(e)
                pass  

In [12]:
ts = time_stamp()
scrape_pages_load_more('https://www.npr.org/sections/news/archive', 'NPR', ts)#, existing_set = True)

/tmp/ipykernel_6108/602359904.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='/home/stephbuon/projects/entascope/geckodriver', options=options)


In [1]:
#ts = time_stamp()
#scrape_pages_multi_page('https://www.kristv.com/?00000169-7dca-d05d-ab6f-7fdbf8990001-page=', 'KRISTV', ts)
# its not under us, its under politics 
#scrape_pages('https://www.foxnews.com/politics', 'FOX', ts)